In [132]:
import pandas as pd

In [133]:
df = pd.read_excel('C:/Users/HP/Desktop/Datascience/Data_Exploration_and_Cleaning/Project-Data-Set-Repository-master (1)/default_of_credit_card_clients.xls')

In [134]:
import numpy as np

In [135]:
np.random.seed(seed=24)


In [136]:
random_integers = np.random.randint(low=1,high=5,size=100)


In [137]:
is_equal_to_3 = random_integers == 3


In [138]:
is_equal_to_3[:5]


array([ True, False, False, False, False])

In [139]:
sum(is_equal_to_3)


22

In [140]:
random_integers[is_equal_to_3]


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [141]:
id_counts = df['ID'].value_counts()
dupe_mask = id_counts == 2
dupe_mask[0:5]

ffa0f012-3bd7    True
9a75e346-c849    True
d5aeb496-64e5    True
3393c9e4-3345    True
c89b26ff-ec7f    True
Name: ID, dtype: bool

In [142]:
dupe_ids = id_counts.index[dupe_mask]
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

In [143]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:].head(10) 


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
541,d5aeb496-64e5,150000,2,1,2,28,1,-2,-2,-2,...,0,0,318,0,0,0,0,318,0,0
641,d5aeb496-64e5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2005,ffa0f012-3bd7,20000,1,2,2,34,Not available,-2,-2,-2,...,2280,1500,780,1530,750,1140,3000,780,0,0
2105,ffa0f012-3bd7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,9a75e346-c849,70000,2,2,1,49,Not available,0,0,0,...,58246,48859,46688,3700,3100,2200,1600,1850,1800,1
19793,9a75e346-c849,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
df_zero_mask = df == 0


In [145]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
sum(feature_zero_mask)


315

In [146]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()


In [147]:
df_clean_1['ID'].nunique()


29685

In [148]:
valid_pay_1_mask = df_clean_1['PAY_1'] != 'Not available'
valid_pay_1_mask[0:5]

0    True
1    True
2    True
3    True
4    True
Name: PAY_1, dtype: bool

In [149]:
df_clean_2 = df_clean_1.loc[valid_pay_1_mask,:].copy()


In [150]:
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')
df_clean_2[['PAY_1', 'PAY_2']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26664 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   PAY_1   26664 non-null  int64
 1   PAY_2   26664 non-null  int64
dtypes: int64(2)
memory usage: 624.9 KB


In [151]:
df_clean_2['EDUCATION'].replace(to_replace=[0, 5, 6], value=4, inplace=True)
df_clean_2['EDUCATION'].value_counts()

2    12458
1     9412
3     4380
4      414
Name: EDUCATION, dtype: int64

In [152]:
df_clean_2['MARRIAGE'].replace(to_replace=0, value=3, inplace=True)
df_clean_2['MARRIAGE'].value_counts()

2    14158
1    12172
3      334
Name: MARRIAGE, dtype: int64

In [153]:
df_clean_2['EDUCATION_CAT'] = 'none'


In [154]:
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)


,EDUCATION,EDUCATION_CAT
0,2,none
1,2,none
2,2,none
3,2,none
4,2,none
5,1,none
6,1,none
7,2,none
8,3,none
9,3,none


In [155]:
cat_mapping = {
    1: "graduate school",
    2: "university",
    3: "high school",
    4: "others"
}

In [156]:

df_clean_2['EDUCATION_CAT'] = df_clean_2['EDUCATION'].map(cat_mapping)
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)

,EDUCATION,EDUCATION_CAT
0,2,university
1,2,university
2,2,university
3,2,university
4,2,university
5,1,graduate school
6,1,graduate school
7,2,university
8,3,high school
9,3,high school


In [157]:
edu_ohe = pd.get_dummies(df_clean_2['EDUCATION_CAT'])
edu_ohe.head(10)
df_with_ohe = pd.concat([df_clean_2, edu_ohe], axis=1)
df_with_ohe[['EDUCATION_CAT', 'graduate school',
             'high school', 'university', 'others']].head(10)
df_with_ohe.to_csv('cleaned_data.csv', index=False)
